title 2

In [20]:
import numpy as np
import pandas as pd
import scipy
from numpy import linalg
import cvxopt
from cvxopt import solvers, matrix
from scipy.spatial.distance import pdist, cdist, squareform

def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=1):
    return np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

def gaussian_kernel_matrix(X, gamma=0.5):
    pairwise_dists = squareform(pdist(X, 'euclidean'))
    K = scipy.exp(-gamma*pairwise_dists ** 2)
    return K

class SVM:
    def __init__(self, C=1, kernel='rbf', gamma=0.1):
        self.C = C
        self.kernel = kernel # kernel_function 'rbf', 'linear'
        self.gamma = gamma # Kernel coefficient gamma for 'rbf'
        
    def fit(self, X, y, mode='OVA', loss='hinge_loss', c=0, degree=2):
        self.loss = loss
        self.mode = mode
        self.c = c # Intercept of the polynomial kernel
        self.degree = degree # Degree of the polynomial kernel
        self.X_train_ = X
        self.n_sample_ = y.shape[0] # n_sample
        self.classes_ = np.unique(y)
        self.alphas_ = {}
        self.K_ = self.fit_kernel()
        if mode == 'OVA':
            for class_ in self.classes_:
                y_copy = y.copy()
                y_copy[y_copy != class_] = -1
                y_copy[y_copy == class_] = 1
                self.fit_dual(y_copy)
                sol = solvers.qp(matrix(self.K_), matrix(self.p_), matrix(self.G_), matrix(self.h_))
                self.alphas_[class_] = np.array(sol['x']).reshape(-1,)
                
        return self
        
    def predict(self, X_test):
        
        predictions = {}
        self.K_test_ = self.fit_kernel_test(X_test)
        self.n_test_ = X_test.shape[0] # size of the test sample
        n = self.n_test_
        res_mat = np.empty((self.classes_.shape[0], n))

        for class_ in self.classes_:
            alpha = self.alphas_[class_]
            res_mat[class_] = np.dot(self.K_test_, alpha)
            #res_mat[class_] = np.sum(alpha*self.K_test_, axis=1) # equivalent to np.dot(alpha, K) ?
        y_pred = res_mat.argmax(axis=0)
        return y_pred  
    """def fit_kernel_test(self, X, ):
        if self.kernel == 'rbf':
            cdist(X, )
        """    
    
    def fit_kernel(self):
        
        X = self.X_train_
        
        if self.kernel == 'rbf':
            pairwise_dists = squareform(pdist(X, 'euclidean'))
            K = scipy.exp(-self.gamma*pairwise_dists ** 2)
            return K
        
        elif self.kernel == 'linear':
            # In fact it's not a kernel
            K = squareform(pdist(X, 'minkowski', 1))
            return K
        
        elif self.kernel == 'polynomial':
            K = (self.c + np.dot(X, X.transpose())) ** self.degree
            return K
        else:
            raise Exception('the kernel must either be rbf or linear')
    
    def fit_kernel_test(self, X_test):
            
        if self.kernel == 'rbf':
            pairwise_dists = cdist(X_test, self.X_train_)
            K = scipy.exp(-self.gamma*pairwise_dists ** 2)
            return K
        
        elif self.kernel == 'linear':
            # In fact it's not a kernel, bu
            K = squareform(cdist(X, self.X_train_, 'minkowski', 1))
            return K
        elif self.kernel == 'polynomial':
            K = (self.c + np.dot(X_test, self.X_train_.transpose())) ** self.degree 
            return K
        else:
            raise Exception('the kernel must either be rbf or linear')
    
    def fit_kernel2(self, X):
            
        if self.kernel == 'rbf':
            pairwise_dists = cdist(X, X)
            K = scipy.exp(-self.gamma*pairwise_dists ** 2)
            return K
        
        elif self.kernel == 'linear':
            # In fact it's not a kernel
            K = squareform(pdist(X, 'minkowski', 1))
            return K
        
        else:
            raise Exception('the kernel must either be rbf or linear')
         
    def fit_dual(self, y):
        
        if self.loss == 'hinge_loss':
            n = self.n_sample_
            diag_y = np.diag(y)
            self.p_ = (-y)
            self.Q_ = self.K_ # Quadratic matrix
            self.G_ = np.r_[diag_y, -diag_y] # Constraint matrix of size(2*n, n)
            self.h_ = np.r_[self.C*np.ones(n), np.zeros(n)]
        elif self.loss == 'squared_hinge_loss':
            """"""
            
        else:
            raise Exception('loss should be hinge loss or squared_hinge_loss')
        
        return self    

In [10]:
%%time
df_X_train = pd.read_csv('Xtr.csv', header=None, usecols=np.arange(3072))
df_X_test = pd.read_csv('Xte.csv', header=None, usecols=np.arange(3072))
df_y_train = pd.read_csv('Ytr.csv')


X_train = np.array(df_X_train, dtype=float)
X_test = np.array(df_X_test, dtype=float)
y_train = np.array(df_y_train['Prediction'], dtype=float)

CPU times: user 4.61 s, sys: 92 ms, total: 4.7 s
Wall time: 4.71 s


In [4]:
"""%%time
# Loading Data after augmentation

X_train_rgb = np.load('X_train_rgb.npy')
y_train_rgb = np.load('y_train_rgb.npy')


X_train_gray = np.load('X_train_gray.npy')
y_train_gray = np.load('y_train_gray.npy')

# Don't forget to shuffle

# Mixing the data after augmentation
idx = 4000
stop = 16000
X = np.r_[X_train[:idx], X_train_rgb[2:stop:8], X_train_rgb[5:stop:8]]
X_val = X_train[idx:]
y = np.r_[y_train[:idx], y_train_rgb[2:stop:8], y_train_rgb[5:stop:8]]
y_val = y_train[idx:]"""


CPU times: user 8 ms, sys: 2.28 s, total: 2.29 s
Wall time: 29.4 s


In [ ]:
c_grid = [0, 1]
degrees = [2, 3, 4]
scores = {}
y_preds = {}
idx = 4500
X = X_train[:idx]
X_val = X_train[idx:]
y = y_train[:idx]
y_val = y_train[idx:]
for c in c_grid:
    for degree in degrees:
        print(degree)
        
        svm = SVM(kernel='polynomial')
        svm.fit(X, y, c=c, degree=degree)
        y_pred = svm.predict(X_val)
        y_preds[degree] = y_pred
        score = np.mean( y_pred != y_val )
        scores[degree] = score
        print(score)

2
     pcost       dcost       gap    pres   dres
 0: -5.1317e+01 -5.2244e+03  2e+04  1e+00  3e-15
 1:  1.8186e-01 -2.0055e+03  3e+03  8e-02  2e-15
 2: -5.0145e+01 -6.1326e+02  6e+02  1e-02  3e-15
 3: -8.5118e+01 -1.5584e+02  7e+01  1e-03  3e-15
 4: -9.0843e+01 -9.8732e+01  8e+00  9e-05  2e-15
 5: -9.1611e+01 -9.2566e+01  1e+00  8e-06  2e-15
 6: -9.1748e+01 -9.1817e+01  7e-02  4e-07  2e-15
 7: -9.1765e+01 -9.1768e+01  3e-03  8e-09  2e-15
 8: -9.1766e+01 -9.1766e+01  2e-04  2e-10  2e-15
 9: -9.1766e+01 -9.1766e+01  9e-06  2e-12  2e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.1323e+01 -5.1715e+03  2e+04  1e+00  3e-15
 1:  2.6300e+01 -1.9763e+03  3e+03  9e-02  2e-15
 2: -2.5327e+01 -6.3244e+02  7e+02  2e-02  2e-15
 3: -6.4565e+01 -1.5625e+02  9e+01  2e-03  5e-15
 4: -7.2009e+01 -8.3528e+01  1e+01  2e-04  2e-15
 5: -7.3054e+01 -7.4110e+01  1e+00  1e-05  2e-15
 6: -7.3216e+01 -7.3309e+01  9e-02  5e-07  2e-15
 7: -7.3237e+01 -7.3243e+01  5e-03  2e-08  2e

In [34]:
y_preds[2]-y_val


array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [21]:
%%time
idx = 2000
X = X_train[:idx]
X_val = X_train[idx:idx+1000]
y = y_train[:idx]
y_val = y_train[idx:idx+1000]
svm = SVM(kernel='polynomial')
svm.fit(X, y, )

     pcost       dcost       gap    pres   dres
 0: -2.3229e+01 -2.3346e+03  9e+03  1e+00  2e-15
 1:  4.5304e+00 -9.0520e+02  1e+03  7e-02  1e-15
 2: -2.7998e+01 -2.2559e+02  2e+02  8e-03  2e-15
 3: -4.3632e+01 -6.1881e+01  2e+01  4e-04  2e-15
 4: -4.5003e+01 -4.7483e+01  2e+00  4e-05  1e-15
 5: -4.5150e+01 -4.5335e+01  2e-01  2e-06  1e-15
 6: -4.5168e+01 -4.5175e+01  7e-03  5e-08  1e-15
 7: -4.5170e+01 -4.5170e+01  3e-04  6e-10  1e-15
 8: -4.5170e+01 -4.5170e+01  1e-05  7e-12  1e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.5991e+01 -2.3241e+03  9e+03  1e+00  2e-15
 1:  1.4525e+01 -8.9959e+02  1e+03  7e-02  2e-15
 2: -2.1320e+01 -1.9278e+02  2e+02  6e-03  2e-15
 3: -3.7305e+01 -5.3471e+01  2e+01  3e-04  4e-15
 4: -3.8624e+01 -4.0530e+01  2e+00  3e-05  1e-15
 5: -3.8747e+01 -3.8850e+01  1e-01  5e-07  1e-15
 6: -3.8759e+01 -3.8764e+01  5e-03  1e-08  1e-15
 7: -3.8760e+01 -3.8761e+01  2e-04  2e-10  1e-15
 8: -3.8760e+01 -3.8760e+01  1e-05  2e-12  1e-1

In [22]:
y_pred = svm.predict(X_val)


/root/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:60: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [23]:
np.mean(y_val != y_pred)

0.752

Short test with n=2000
* degree = 4 accuracy 16%
* degree = 3 accuracy 20% 
* degree = 2 accuracy 20%

In [72]:
gammas = [0.01, 0.1, 1]
scores = {}
y_preds = {}
for gamma in gammas:
    svm = SVM(gamma=gamma)
    svm.fit(X, y)
    y_pred = svm.predict(X_val)
    scores[gamma] = np.mean(y_pred != y_val)
    y_preds[gamma] = y_pred

(4000,)
     pcost       dcost       gap    pres   dres
 0: -1.3877e+03 -8.9315e+03  4e+04  3e+00  3e-14
 1: -8.4427e+02 -5.1402e+03  5e+03  9e-02  3e-14
 2: -8.0597e+02 -1.0090e+03  2e+02  9e-04  2e-14
 3: -8.1113e+02 -8.9645e+02  9e+01  2e-04  2e-14
 4: -8.1628e+02 -8.6170e+02  5e+01  8e-05  2e-14
 5: -8.2051e+02 -8.3584e+02  2e+01  2e-05  2e-14
 6: -8.2294e+02 -8.2833e+02  5e+00  3e-06  2e-14
 7: -8.2404e+02 -8.2560e+02  2e+00  2e-07  2e-14
 8: -8.2442e+02 -8.2469e+02  3e-01  7e-09  2e-14
 9: -8.2450e+02 -8.2453e+02  3e-02  4e-10  2e-14
10: -8.2451e+02 -8.2451e+02  1e-03  1e-11  3e-14
11: -8.2451e+02 -8.2451e+02  3e-05  3e-13  2e-14
Optimal solution found.
(4000,)
     pcost       dcost       gap    pres   dres
 0: -1.3045e+03 -9.2105e+03  5e+04  3e+00  4e-14
 1: -8.2720e+02 -5.3825e+03  6e+03  1e-01  4e-14
 2: -7.9900e+02 -1.5370e+03  8e+02  1e-02  3e-14
 3: -8.0370e+02 -1.1465e+03  3e+02  3e-03  2e-14
 4: -8.1731e+02 -9.3337e+02  1e+02  6e-04  2e-14
 5: -8.2569e+02 -8.7283e+02  5e

/root/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:58: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


(4000,)
     pcost       dcost       gap    pres   dres
 0: -1.0863e+03 -9.4918e+03  5e+04  3e+00  1e-14
 1: -7.6957e+02 -5.7246e+03  7e+03  2e-01  1e-14
 2: -7.3443e+02 -1.2093e+03  5e+02  5e-03  9e-15
 3: -7.6227e+02 -9.3249e+02  2e+02  6e-04  8e-15
 4: -7.8015e+02 -8.2348e+02  4e+01  9e-05  9e-15
 5: -7.8722e+02 -7.9707e+02  1e+01  1e-05  9e-15
 6: -7.8934e+02 -7.9089e+02  2e+00  1e-06  1e-14
 7: -7.8977e+02 -7.8989e+02  1e-01  8e-08  1e-14
 8: -7.8981e+02 -7.8981e+02  7e-03  4e-09  1e-14
 9: -7.8981e+02 -7.8981e+02  2e-04  6e-11  1e-14
Optimal solution found.
(4000,)
     pcost       dcost       gap    pres   dres
 0: -8.5754e+02 -8.5578e+03  4e+04  3e+00  1e-14
 1: -6.4990e+02 -4.8934e+03  6e+03  1e-01  1e-14
 2: -6.3658e+02 -1.5245e+03  1e+03  2e-02  1e-14
 3: -6.7004e+02 -9.3952e+02  3e+02  3e-03  1e-14
 4: -6.9653e+02 -7.6172e+02  7e+01  2e-04  1e-14
 5: -7.0705e+02 -7.2328e+02  2e+01  1e-05  1e-14
 6: -7.1062e+02 -7.1321e+02  3e+00  2e-16  1e-14
 7: -7.1134e+02 -7.1164e+02  3e

short CV on gamma lead to : {0.01: 0.80900000000000005, 0.1: 0.77400000000000002, 1: 0.875}

In [23]:
%%time
X_train = np.array(df_X_train)
X_test = np.array(df_X_test)
y_train = np.array(df_y_train['Prediction'])

CPU times: user 12 ms, sys: 24 ms, total: 36 ms
Wall time: 35.8 ms


In [5]:
def computing_gram_matrix3(X, sigma=0.5):
    f = lambda x, y: np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))
    return squareform(pdist(X, f))

In [32]:
import scipy
from scipy.spatial.distance import pdist, squareform

def computing_gram_matrix(X, sigma=1):
    n, p = X.shape
    K = np.zeros((n,n))
    for ii in range(n):
        for kk in range(n):
            K[ii, kk] = np.exp(-linalg.norm(X[ii]-X[kk])**2 / (2 * (sigma ** 2)))
    return K

def computing_gram_matrix2(X, sigma=1):
    pairwise_dists = squareform(pdist(X, 'euclidean'))
    K = scipy.exp(-pairwise_dists ** 2 / (2*sigma**2))
    return K
def computing_gram_matrix3(X, sigma=1):
    pairwise_dists = cdist(X, X)
    K = scipy.exp(-pairwise_dists ** 2 / (2*sigma**2))
    return K

In [7]:
%%time
K1 = computing_gram_matrix(X_train)

CPU times: user 10min 22s, sys: 64 ms, total: 10min 22s
Wall time: 10min 22s


In [33]:
%%time
K2=computing_gram_matrix3(X_train)

CPU times: user 1min 52s, sys: 140 ms, total: 1min 52s
Wall time: 1min 53s


In [35]:
K2

array([[ 1.        ,  0.00866644,  0.02227502, ...,  0.0104907 ,
         0.02156173,  0.01812238],
       [ 0.00866644,  1.        ,  0.00345238, ...,  0.00275313,
         0.00322442,  0.00347302],
       [ 0.02227502,  0.00345238,  1.        , ...,  0.00580637,
         0.0075947 ,  0.01033267],
       ..., 
       [ 0.0104907 ,  0.00275313,  0.00580637, ...,  1.        ,
         0.0033863 ,  0.00503564],
       [ 0.02156173,  0.00322442,  0.0075947 , ...,  0.0033863 ,
         1.        ,  0.00542766],
       [ 0.01812238,  0.00347302,  0.01033267, ...,  0.00503564,
         0.00542766,  1.        ]])

In [11]:
def __init__(self, penalty='l2', loss='squared_hinge', dual=True, tol=1e-4,
                 C=1.0, multi_class='ovr', fit_intercept=True,
                 intercept_scaling=1, class_weight=None, verbose=0,
                 random_state=None, max_iter=1000):
    self.dual = dual
    self.tol = tol
    self.C = C
    self.multi_class = multi_class
    self.fit_intercept = fit_intercept
    self.intercept_scaling = intercept_scaling
    self.class_weight = class_weight
    self.verbose = verbose
    self.random_state = random_state
    self.max_iter = max_iter
    self.penalty = penalty
    self.loss = loss

def fit(self, X, y, sample_weight=None):
    """Fit the model according to the given training data.
    Parameters
    ----------
    X : {array-like, sparse matrix}, shape = [n_samples, n_features]
        Training vector, where n_samples in the number of samples and
        n_features is the number of features.
    y : array-like, shape = [n_samples]
        Target vector relative to X
    sample_weight : array-like, shape = [n_samples], optional
        Array of weights that are assigned to individual
        samples. If not provided,
        then each sample is given unit weight.
    Returns
    -------
    self : object
        Returns self.
    """
    # FIXME Remove l1/l2 support in 1.0 -----------------------------------
    msg = ("loss='%s' has been deprecated in favor of "
           "loss='%s' as of 0.16. Backward compatibility"
           " for the loss='%s' will be removed in %s")

    if self.loss in ('l1', 'l2'):
        old_loss = self.loss
        self.loss = {'l1': 'hinge', 'l2': 'squared_hinge'}.get(self.loss)
        warnings.warn(msg % (old_loss, self.loss, old_loss, '1.0'),
                      DeprecationWarning)
    # ---------------------------------------------------------------------

    if self.C < 0:
        raise ValueError("Penalty term must be positive; got (C=%r)"
                         % self.C)

    X, y = check_X_y(X, y, accept_sparse='csr',
                     dtype=np.float64, order="C")
    check_classification_targets(y)
    self.classes_ = np.unique(y)

    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
        X, y, self.C, self.fit_intercept, self.intercept_scaling,
        self.class_weight, self.penalty, self.dual, self.verbose,
        self.max_iter, self.tol, self.random_state, self.multi_class,
        self.loss, sample_weight=sample_weight)

    if self.multi_class == "crammer_singer" and len(self.classes_) == 2:
        self.coef_ = (self.coef_[1] - self.coef_[0]).reshape(1, -1)
        if self.fit_intercept:
            intercept = self.intercept_[1] - self.intercept_[0]
            self.intercept_ = np.array([intercept])

    return self

array([[ 1.        ,  0.00866644,  0.02227502, ...,  0.0104907 ,
         0.02156173,  0.01812238],
       [ 0.00866644,  1.        ,  0.00345238, ...,  0.00275313,
         0.00322442,  0.00347302],
       [ 0.02227502,  0.00345238,  1.        , ...,  0.00580637,
         0.0075947 ,  0.01033267],
       ..., 
       [ 0.0104907 ,  0.00275313,  0.00580637, ...,  1.        ,
         0.0033863 ,  0.00503564],
       [ 0.02156173,  0.00322442,  0.0075947 , ...,  0.0033863 ,
         1.        ,  0.00542766],
       [ 0.01812238,  0.00347302,  0.01033267, ...,  0.00503564,
         0.00542766,  1.        ]])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()